In [6]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re
from selenium.webdriver.common.by import By

In [ ]:
def insta_searching(word):
    url = "https://www.instagram.com/explore/tags/" + str(word)
    return url
def select_first(driver):
    first = driver.find_elements(By.CSS_SELECTOR, "div._aagw")[0]
    first.click()
    time.sleep(3)
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    # 본문 내용
    try:
        content = soup.select('div._a9zs')[0].text
    except:
        content = ''
    # 해시태그
    tags = re.findall(r'#[^\s#,\\]+', content)
    
    # 작성일자
    date = soup.select('time._aaqe')[0]['datetime'][:10]
    
    # 좋아요
    try:
        like = soup.select('div._aacl._aaco._aacw._aacx._aada._aade')[0].findAll('span')[-1].text
    except:
        like = 0
    # 위치
    try:
        place = soup.select('div._aaqm')[0].text
    except:
        place = ''

    data = [content, date, like, place, tags]
    return data
def move_next(driver):
    right = driver.find_element(By.CSS_SELECTOR, "div._aaqg._aaqh") # 2022/01/11 수정
    right.click()
    time.sleep(3)
# 크롤링 시작
"""
driver.get(url)을 통해 검색 페이지 접속하고,
target 변수에 크롤링할 게시글의 수를 바인딩
"""

# 크롬 브라우저 열기
driver = webdriver.Chrome("chromedriver.exe")

driver.get('https://www.instagram.com')
time.sleep(3)

# 인스타그램 로그인을 위한 계정 정보
# email = "__hiksang__siksang__"
# input_id = driver.find_elements(By.CSS_SELECTOR,'input._2hvTZ.pexuQ.zyHYP')[0]
# input_id.clear()
# input_id.send_keys(email)

# password = "thdusdl1!"
# input_pw = driver.find_elements(By.CSS_SELECTOR, 'input._2hvTZ.pexuQ.zyHYP')[1]
# input_pw.clear()
# input_pw.send_keys(password)
# input_pw.submit()

# time.sleep(5)


loginbutton = driver.find_element("xpath", "//*[@id='loginForm']/div/div[3]")
enter_username = driver.find_element("name","username")
password = driver.find_element("name","password")

#CHANGE USERNAME AND PASSWORD WITH YOUR CREDENTIALS.
enter_username.send_keys("kimhiksang")
password.send_keys("rlagmltkd1!")


time.sleep(5)


loginbutton.click()


time.sleep(5)

# 게시물을 조회할 검색 키워드 입력 요청
word = input("검색어를 입력하세요 : ")
word = str(word)
url = insta_searching(word)

C:\Users\Gachon\AppData\Local\Temp\ipykernel_20588\1733326898.py:46: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe")


In [ ]:
# 검색 결과 페이지 열기
driver.get(url)
time.sleep(10) # 코드 수행 환경에 따라 페이지가 로드되는 데 시간이 더 걸릴 수 있어 8초로 변경(2022/01/11)

# 첫 번째 게시물 클릭
select_first(driver)

# 본격적으로 데이터 수집 시작
results = []
## 수집할 게시물의 수
target = 500
for i in range(target):

    try:
        data = get_content(driver)
        results.append(data)
        move_next(driver)
    except:
        time.sleep(2)
        move_next(driver)
    time.sleep(5)

In [ ]:
import pandas as pd
from datetime import datetime

date = datetime.today().strftime('%Y-%m-%d')

results_df = pd.DataFrame(results)
results_df.columns = ['content','date','like','place','tags']
results_df.to_excel(date + '_about '+word+' insta crawling.xlsx')